# Extraction

Reads the three pre-filtered country files:

- `{COUNTRY}-areas.geo.jsonseq`
- `{COUNTRY}-streets.geo.jsonseq`
- `{COUNTRY}-housenums.geo.jsonseq`

And process it as follows:

- Extract only certain properties
- Country border clipping
- For areas: keep only administrative and postal_code boundaries
- For streets: keep only those with a name
- For housenums: compute centroid

The result are 2 "final" tab-separated-values files:

**postal-codes.tsv**

    - postal_code
    - geometry: MultiPolygon

**administrative-areas.tsv**

    - name
    - name:en
    - alt_name
    - geometry: MultiPolygon

And 2 "temporary" tab-separated-values files, to be refined in the next process.

**streets.tsv**

    - name
    - ?name:en
    - ?alt_name
    - geometry: LineString or MultiPolygon

**housenums.tsv**

    - housenumber
    - ?street
    - ?postcode
    - ?city
    - geometry: Point (centroid)


The tags to extract from the full geojson dumps.
They vary strongly from country to country or even within different regions of a country.

The site https://taginfo.openstreetmap.org/tags provides good insight on available tags and their usage.

In [ ]:
import os

COUNTRY = os.environ.get('COUNTRY', 'andorra')
COUNTRY_CODE = os.environ.get('CODE', 'AD')

AREA_TAGS = [
    'id',
    'boundary',
    'admin_level',
    'postal_code',
    'name',
    'alt_name',
    'name:en',
]


STREET_TAGS = [
    'id',
    'highway',
    'name',
    'alt_name',
]

HOUSE_TAGS = [
    'id',
    'addr:housenumber',
    'addr:street',
    'addr:city',
    'addr:postcode'
]

In [ ]:
import pandas as pd

admin_level_mapping = pd.read_csv('admin_level_mapping.tsv', sep='\t', dtype={'admin_level':str})
admin_level_mapping = admin_level_mapping[admin_level_mapping['country_code'] == COUNTRY_CODE]
admin_level_mapping = admin_level_mapping.set_index('admin_level')['area_type']

admin_level_mapping

# Areas

In [ ]:
import geopandas as gpd
areas = gpd.read_file(f'temp/{COUNTRY}-areas.geo.jsonseq', encoding="utf8", engine="pyogrio", columns=AREA_TAGS)
areas.rename(columns={'id': 'area_id', 'name': 'area_name'}, inplace=True)
areas


In [ ]:
areas['geometry'].type.value_counts()

## Country borders

There might be multiple boundaries, for example because of enclaves in neighboring countries or islands beyond territorial waters.

In [ ]:
country_borders = areas[areas['admin_level'] == '2']
country_borders.boundary.plot()

## Filtering

In [ ]:
areas = areas[areas['boundary'].isin(['administrative', 'postal_code'])]
areas

In [ ]:
areas = areas.clip(country_borders)
areas

In [ ]:
areas['geometry'].type.value_counts()

In [ ]:
areas = areas[areas['geometry'].type.isin(['Polygon', 'MultiPolygon'])]
areas

In [ ]:
# copy() to release memory instead of slice which keeps the original ...does mem release work?
areas = areas.copy()

## Administrative areas

In [ ]:
for i in range(3,11):
    level = str(i)
    admin_level = areas[areas['admin_level'] == level]
    if not admin_level.empty:
        area_name = admin_level_mapping.get(level, '??? (ignored)')
        ax = admin_level.plot()
        country_borders.boundary.plot(linewidth=0.5, ax=ax, color='green')
        ax.set_title(f'{COUNTRY.capitalize()} - level {i} - {area_name}')

In [ ]:
administrative_areas = areas.drop(columns=['boundary', 'postal_code']).dropna(subset='admin_level').sort_values(['admin_level', 'area_name'])
administrative_areas['admin_level'].replace(admin_level_mapping, inplace=True)
administrative_areas.to_csv(f'addresses/{COUNTRY}-administrative-areas.tsv', index=False, encoding="utf8", sep='\t')
administrative_areas

## Postal codes

In [ ]:
postal_codes = areas[['area_id', 'postal_code', 'geometry']].dropna().sort_values(['postal_code']).copy()
postal_codes.to_csv(f'addresses/{COUNTRY}-postal-codes.tsv', index=False, encoding="utf8", sep='\t')
postal_codes

In [ ]:
ax = postal_codes.plot()
country_borders.boundary.plot(linewidth=0.5, ax=ax, color='green')

# Streets

In [ ]:
streets = gpd.read_file(f'temp/{COUNTRY}-streets.geo.jsonseq', encoding="utf8", engine="pyogrio", columns=STREET_TAGS)
streets.rename(columns={'id': 'street_id', 'name': 'street_name'}, inplace=True)
streets

## Filtering

In [ ]:
# 'footway' is included because in "less urbanized" places, there are valid addresses on footways 
HIGHWAY_TYPES = ['trunk', 'primary', 'secondary', 'tertiary', 'unclassified', 'residential', 'living_street', 'service', 'pedestrian', 'track', 'road', 'footway']
streets = streets[streets['highway'].isin(HIGHWAY_TYPES)]
streets

In [ ]:
print("Percentage of tags present in raw data")
(1-streets.isna().mean().round(3)) * 100

In [ ]:
# copy() to release memory instead of slice which keeps the original ...does mem release work?
streets = streets.copy()

## Mapping areas to streets

In [ ]:
# Using the "centroid" workaround for performance reasons
# See https://github.com/geopandas/geopandas/issues/2840
# Note that the full "way" geometry will be needed later
streets['center'] = streets['geometry'].centroid
streets.set_geometry('center', inplace=True)
streets = streets.clip(country_borders).copy()
streets

In [ ]:
from tqdm import tqdm

#streets.set_geometry('geometry', inplace=True) # More complete
streets.set_geometry('center', inplace=True) # Quicker
#streets.drop(columns='index_right', inplace=True)

for admin_level in tqdm(admin_level_mapping.unique()):
    print(admin_level)
    if admin_level == 'country':
        continue
    aa = administrative_areas[administrative_areas['admin_level'] == admin_level][['area_name', 'geometry']]
    aa = aa.rename(columns={'area_name': admin_level})
    streets = streets.sjoin(aa, predicate='intersects', how='left').drop(columns='index_right')

streets

In [ ]:
streets = streets.sjoin(postal_codes[['postal_code','geometry']], how='left').drop(columns='index_right')
streets

## Merging multi-section streets

In [ ]:
streets.set_geometry('geometry', inplace=True)
streets.drop(columns=['center'], inplace=True)
props = [c for c in streets.columns if c not in STREET_TAGS + ['street_id']]
streets[props]

In [ ]:
#def agg(col):
#    return ';'.join(col.unique())
#streets.dissolve(by=['street_name', 'admin_level_7', 'admin_level_8'], aggfunc=agg)


In [ ]:
streets['country'] = COUNTRY_CODE
props = [c for c in admin_level_mapping.unique()] + ['postal_code', 'street_name']
streets_tsv = streets[props].drop_duplicates()
streets_tsv.sort_values(by=list(streets_tsv.columns), inplace=True)
streets_tsv

In [ ]:
streets_tsv.to_csv(f'addresses/{COUNTRY}-streets.tsv.gz', sep='\t', index=False)
streets_tsv = None

# House numbers

In [ ]:
housenums = gpd.read_file(f'temp/{COUNTRY}-housenums.geo.jsonseq', encoding="utf8", engine="pyogrio", columns=HOUSE_TAGS)
housenums.rename(columns={'id': 'house_id'}, inplace=True)
#housenums.set_index('house_id', inplace=True)
housenums

In [ ]:
print("Percentage of tags present in raw data")
(1-housenums.isna().mean().round(3)) * 100

In [ ]:
housenums['geometry'].type.value_counts()

In [ ]:
housenums['geometry'] = housenums['geometry'].centroid
housenums = housenums.clip(country_borders).copy()
housenums

## Merge house duplicates

Often, house polygons include an additonal house "point".

## Finding missing streets

Finding the nearest street of a house is an *expensive computation*. Therefore, we will work on a *slice* of the dataset having no `addr:street` tag.

In [ ]:
street_names = streets[['street_name', 'geometry']]
street_names

In [ ]:
street_names.set_geometry('geometry', inplace=True) # More complete

missing_house_streets = housenums[housenums['addr:street'].isna()]
missing_house_streets = missing_house_streets.to_crs(3857).sjoin_nearest(street_names.to_crs(3857), max_distance=100)
missing_house_streets

In [ ]:
# List of houses where no street within 100m was found
missing_house_streets[missing_house_streets['street_name'].isna()]

In [ ]:
housenums['street'] = housenums['addr:street']
housenums['street'][missing_house_streets.index] = missing_house_streets['street_name']
housenums

## Mapping areas to houses

In [ ]:
from tqdm import tqdm

streets.set_geometry('geometry', inplace=True) # More complete

for admin_level in tqdm(administrative_areas['admin_level'].unique()):
    print(admin_level)
    if admin_level == 'country':
        continue
    aa = administrative_areas[administrative_areas['admin_level'] == admin_level][['area_name', 'geometry']]
    aa = aa.rename(columns={'area_name': admin_level})
    housenums = housenums.sjoin(aa, predicate='intersects', how='left').drop(columns='index_right')

housenums

In [ ]:
housenums = housenums.sjoin(postal_codes[['postal_code','geometry']], how='left').drop(columns='index_right')
housenums

In [ ]:
print("Percentage of tags present in raw data")
(1-housenums.isna().mean().round(3)) * 100

In [ ]:
housenums.dropna(subset=['street'], inplace=True)
housenums['x'] = housenums['geometry'].x
housenums['y'] = housenums['geometry'].y
housenums

In [ ]:
housenums['house_number'] = housenums['addr:housenumber']
housenums['city'] = housenums['city'].combine_first(housenums['addr:city'])
housenums['postal_code'] = housenums['postal_code'].combine_first(housenums['addr:postcode'])

In [ ]:
housenums_tsv = housenums[['postal_code', 'city', 'street', 'house_number', 'x', 'y']].drop_duplicates()
housenums_tsv.sort_values(by=list(housenums_tsv.columns), inplace=True)
housenums_tsv['country'] = COUNTRY_CODE
housenums_tsv

In [ ]:
housenums_tsv.to_csv(f'addresses/{COUNTRY}-houses.tsv.gz', sep='\t', index=False)
housenums_tsv = None

# Maps

In [ ]:
(minx, miny, maxx, maxy) = country_borders.total_bounds
dx = 5 * (maxx - minx)
dy = 5 * (maxy - miny)
print((dx,dy))


In [ ]:
import matplotlib.pyplot as plt

ax = country_borders.to_crs(epsg=3857).boundary.plot(linewidth=0.5, figsize=(dx, dy), color='green')
ax = streets.to_crs(epsg=3857).plot(ax=ax, linewidth=0.5, figsize=(dx, dy), color='orange')
ax = housenums.to_crs(epsg=3857).plot(ax=ax, markersize=0.5, color='red')

ax.set_title(f'{COUNTRY.capitalize()} addresses')

import contextily as cx
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, zoom=10)
ax.set_axis_off()